In [ ]:
# !python3 -m pip install --user sklearn

In [1]:
import h5py
import tensorflow as tf
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Reshape, Concatenate, Add
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.optimizers import Adam, RMSprop

import os, sys, re, codecs

In [2]:
TPU_SERVER = '10.143.23.2'
TPU_PORT = ':8470'
TPU_WORKER = 'grpc://' + TPU_SERVER + TPU_PORT
TPU_WORKER

'grpc://10.143.23.2:8470'

In [2]:
def generate_model(max_features, max_len, embed_size, tg_cate_size):
    inputs = []
    embeddings = []

    inp_product  = Input(shape=(max_len, ))
    embedding = Embedding(max_features, embed_size, input_length=max_len)(inp_product)
    embedding = Reshape(target_shape=(max_len*embed_size,))(embedding)
    embedding = Dense(256, activation='relu')(embedding)
    inputs.append(inp_product)
    embeddings.append(embedding)

    inp_model  = Input(shape=(max_len, ))
    embedding = Embedding(max_features, embed_size, input_length=max_len)(inp_model)
    embedding = Reshape(target_shape=(max_len*embed_size,))(embedding)
    embedding = Dense(256, activation='relu')(embedding)
    inputs.append(inp_model)
    embeddings.append(embedding)

    inp_brand  = Input(shape=(max_len, ))
    embedding = Embedding(max_features, embed_size, input_length=max_len)(inp_brand)
    embedding = Reshape(target_shape=(max_len*embed_size,))(embedding)
    embedding = Dense(256, activation='relu')(embedding)
    inputs.append(inp_brand)
    embeddings.append(embedding)

    inp_maker  = Input(shape=(max_len, ))
    embedding = Embedding(max_features, embed_size, input_length=max_len)(inp_maker)
    embedding = Reshape(target_shape=(max_len*embed_size,))(embedding)
    embedding = Dense(256, activation='relu')(embedding)
    inputs.append(inp_maker)
    embeddings.append(embedding)

    inp_img = Input(shape=(2048, ))
    feat = Dropout(0.4)(inp_img)
    feat = Dense(256, activation='relu')(feat)
    inputs.append(inp_img)
    embeddings.append(feat)

    x = Concatenate()(embeddings)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)

    # decoder1 = add([x, feat])
    # decoder2 = Dense(256, activation='relu')(decoder1)
    output = Dense(tg_cate_size, activation='softmax')(x)

    model = Model(inputs, output)
    # model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

    model.summary()
    
    return model

In [3]:
!free

              total        used        free      shared  buff/cache   available
Mem:      206357056     5363328   116423776       10872    84569952   199144892
Swap:             0           0           0


In [4]:
def data_generator(list_X, y, batch_size, total_size):
    start_idx = 0
    end_idx = 0
    
    while True:
        if (start_idx + batch_size) >= total_size:
            start_idx = 0
            end_idx = 0
        
        end_idx += batch_size
        batch_X = []
        for X in list_X:
            batch_X.append(X[start_idx:end_idx])
            
        batch_y = np.array(y[start_idx:end_idx])
        start_idx = end_idx        
        
        yield batch_X, batch_y

In [2]:
X_product = h5py.File('origin/X_train_product.h5py', 'r')
X_maker = h5py.File('origin/X_train_maker.h5py', 'r')
X_brand = h5py.File('origin/X_train_brand.h5py', 'r')
X_model = h5py.File('origin/X_train_model.h5py', 'r')
X_image = h5py.File('origin/X_train_img_feat.h5py', 'r')
X_con = h5py.File('origin/X_train_concate.h5py', 'r')

y_b_cate = h5py.File('origin/y_train_bcateid.h5py', 'r')
y_m_cate = h5py.File('origin/y_train_mcateid.h5py', 'r')
y_s_cate = h5py.File('origin/y_train_scateid.h5py', 'r')
y_d_cate = h5py.File('origin/y_train_dcateid.h5py', 'r')

print("Product / train : {}, val : {}".format(X_product['train'].shape, X_product['val'].shape))
print("Maker / train : {}, val : {}".format(X_maker['train'].shape, X_maker['val'].shape))
print("Brand / train : {}, val : {}".format(X_brand['train'].shape, X_brand['val'].shape))
print("Model / train : {}, val : {}".format(X_model['train'].shape, X_model['val'].shape))
print("Image / train : {}, val : {}".format(X_image['train'].shape, X_image['val'].shape))
print("Concate / train : {}, val : {}".format(X_con['train'].shape, X_con['val'].shape))

print("B cate / train : {}, val : {}".format(y_b_cate['train'].shape, y_b_cate['val'].shape))
print("M cate / train : {}, val : {}".format(y_m_cate['train'].shape, y_m_cate['val'].shape))
print("S cate / train : {}, val : {}".format(y_s_cate['train'].shape, y_s_cate['val'].shape))
print("D cate / train : {}, val : {}".format(y_d_cate['train'].shape, y_d_cate['val'].shape))

Product / train : (6507854, 32), val : (1626964, 32)
Maker / train : (6507854, 32), val : (1626964, 32)
Brand / train : (6507854, 32), val : (1626964, 32)
Model / train : (6507854, 32), val : (1626964, 32)
Image / train : (6507854, 2048), val : (1626964, 2048)
Concate / train : (6507854, 32), val : (1626964, 32)
B cate / train : (6507854, 57), val : (1626964, 57)
M cate / train : (6507854, 552), val : (1626964, 552)
S cate / train : (6507854, 3191), val : (1626964, 3191)
D cate / train : (6507854, 405), val : (1626964, 405)


In [3]:
s_train_removed = np.array(y_s_cate['train'][:,0])
s_val_removed = np.array(y_s_cate['val'][:,0])

d_train_removed = np.array(y_d_cate['train'][:,0])
d_val_removed = np.array(y_d_cate['val'][:,0])

print(s_train_removed.shape)
print(s_val_removed.shape)
print(d_train_removed.shape)
print(d_val_removed.shape)

(6507854,)
(1626964,)
(6507854,)
(1626964,)


In [10]:
y_m_tr = y_m_cate['train'][:]
y_m_val = y_m_cate['val'][:]

In [11]:
y_m_tr_rm_s = y_m_tr[s_train_removed[:] == 0]
y_m_val_rm_s = y_m_val[s_val_removed[:] == 0]

print(y_m_tr_rm_s.shape)
print(y_m_val_rm_s.shape)

(5055885, 552)
(1212813, 552)


In [12]:
with h5py.File('./origin/y_mcateid_rm_for_s.h5py', 'w') as write:
    write.create_dataset('train', (5055885, 552), dtype='?')
    write.create_dataset('val', (1212813, 552), dtype='?')
    
    write['train'][:] = y_m_tr_rm_s[:]
    write['val'][:] = y_m_val_rm_s[:]

In [13]:
y_m_tr_rm_d = y_m_tr[d_train_removed[:] == 0]
y_m_val_rm_d = y_m_val[d_val_removed[:] == 0]

print(y_m_tr_rm_d.shape)
print(y_m_val_rm_d.shape)

(607560, 552)
(150332, 552)


In [14]:
with h5py.File('./origin/y_mcateid_rm_for_d.h5py', 'w') as write:
    write.create_dataset('train', (607560, 552), dtype='?')
    write.create_dataset('val', (150332, 552), dtype='?')
    
    write['train'][:] = y_m_tr_rm_d[:]
    write['val'][:] = y_m_val_rm_d[:]

In [15]:
y_b_tr = y_b_cate['train'][:]
y_b_val = y_b_cate['val'][:]

In [16]:
y_b_tr_removed = y_b_tr[s_train_removed[:] == 0]
y_b_val_removed = y_b_val[s_val_removed[:] == 0]

print(y_b_tr_removed.shape)
print(y_b_val_removed.shape)

(5055885, 57)
(1212813, 57)


In [17]:
with h5py.File('./origin/y_bcateid_rm_for_s.h5py', 'w') as write:
    write.create_dataset('train', (5055885, 57), dtype='?')
    write.create_dataset('val', (1212813, 57), dtype='?')
    
    write['train'][:] = y_b_tr_removed[:]
    write['val'][:] = y_b_val_removed[:]

In [18]:
y_b_tr_rm_d = y_b_tr[d_train_removed[:] == 0]
y_b_val_rm_d = y_b_val[d_val_removed[:] == 0]

print(y_b_tr_rm_d.shape)
print(y_b_val_rm_d.shape)

(607560, 57)
(150332, 57)


In [19]:
with h5py.File('./origin/y_bcateid_rm_for_d.h5py', 'w') as write:
    write.create_dataset('train', (607560, 57), dtype='?')
    write.create_dataset('val', (150332, 57), dtype='?')
    
    write['train'][:] = y_b_tr_rm_d[:]
    write['val'][:] = y_b_val_rm_d[:]

In [4]:
X_img_tr = X_image['train'][:]
X_img_val = X_image['val'][:]

In [ ]:
!free

In [25]:
# X_img_train.shape
# X_img_val.shape

NameError: name 'X_img_train' is not defined

In [5]:
X_img_s_tr = X_img_tr[s_train_removed[:] == 0]
# X_img_s_val = X_img_val[s_val_removed[:] == 0]

print(X_img_s_tr.shape)
# print(X_img_s_val.shape)

(5055885, 2048)


In [6]:
with h5py.File('scate/X_image_rm_s.h5py', 'w') as write:
    write.create_dataset('train', (5055885, 2048), dtype='f4')
#     write.create_dataset('val', (1626964, 2048), dtype='f4')
    
    write['train'][:] = X_img_s_tr[:]
#     write['val'][:] = X_img_s_val[:]
    

In [7]:
X_img_d_tr = X_img_tr[d_train_removed[:] == 0]
# X_img_d_val = X_img_val[d_val_removed[:] == 0]

print(X_img_d_tr.shape)
# print(X_img_d_val.shape)

(607560, 2048)


In [8]:
with h5py.File('dcate/X_image_rm_d.h5py', 'w') as write:
    write.create_dataset('train', (607560, 2048), dtype='f4')
#     write.create_dataset('val', (1626964, 2048), dtype='f4')
    
    write['train'][:] = X_img_d_tr[:]
#     write['val'][:] = X_img_d_val[:]
    

In [9]:
# X_img_s_tr = X_img_tr[s_train_removed[:] == 0]
X_img_s_val = X_img_val[s_val_removed[:] == 0]

# print(X_img_s_tr.shape)
print(X_img_s_val.shape)

(1212813, 2048)


In [10]:
with h5py.File('scate/X_image_rm_s.h5py', 'r+') as write:
#     write.create_dataset('train', (6507854, 32), dtype='f4')
    write.create_dataset('val', (1212813, 2048), dtype='f4')
    
#     write['train'][:] = X_img_s_tr[:]
    write['val'][:] = X_img_s_val[:]
    

In [11]:
# X_img_d_tr = X_img_tr[d_train_removed[:] == 0]
X_img_d_val = X_img_val[d_val_removed[:] == 0]

# print(X_img_d_tr.shape)
print(X_img_d_val.shape)

(150332, 2048)


In [12]:
with h5py.File('dcate/X_image_rm_d.h5py', 'r+') as write:
#     write.create_dataset('train', (6507854, 32), dtype='f4')
    write.create_dataset('val', (150332, 2048), dtype='f4')
    
#     write['train'][:] = X_img_d_tr[:]
    write['val'][:] = X_img_d_val[:]
    

In [8]:
y_s_tr = y_s_cate['train'][:]
y_s_val = y_s_cate['val'][:]

In [9]:
y_s_tr_removed = y_s_tr[s_train_removed[:] == 0]
y_s_val_removed = y_s_val[s_val_removed[:] == 0]

print(y_s_tr_removed.shape)
print(y_s_val_removed.shape)

(5055885, 3191)
(1212813, 3191)


In [10]:
with h5py.File('./origin/y_scateid_rm.h5py', 'w') as write:
    write.create_dataset('train', (5055885, 3191), dtype='?')
    write.create_dataset('val', (1212813, 3191), dtype='?')
    
    write['train'][:] = y_s_tr_removed[:]
    write['val'][:] = y_s_val_removed[:]

In [22]:
y_s_tr_rm_d = y_s_tr[d_train_removed[:] == 0]
y_s_val_rm_d = y_s_val[d_val_removed[:] == 0]

print(y_s_tr_rm_d.shape)
print(y_s_val_rm_d.shape)

(607560, 3191)
(150332, 3191)


In [23]:
with h5py.File('./origin/y_scateid_rm_for_d.h5py', 'w') as write:
    write.create_dataset('train', (607560, 3191), dtype='?')
    write.create_dataset('val', (150332, 3191), dtype='?')
    
    write['train'][:] = y_s_tr_rm_d[:]
    write['val'][:] = y_s_val_rm_d[:]

In [4]:
y_d_tr = y_d_cate['train'][:]
y_d_val = y_d_cate['val'][:]

In [5]:
y_d_tr_removed = y_d_tr[d_train_removed[:] == 0]
y_d_val_removed = y_d_val[d_val_removed[:] == 0]

print(y_d_tr_removed.shape)
print(y_d_val_removed.shape)

(607560, 405)
(150332, 405)


In [6]:
with h5py.File('dcate/y_dcateid_rm.h5py', 'w') as write:
    write.create_dataset('train', (607560, 405), dtype='?')
    write.create_dataset('val', (150332, 405), dtype='?')
    
    write['train'][:] = y_d_tr_removed[:]
    write['val'][:] = y_d_val_removed[:]

In [8]:
X_pro_tr = X_product['train'][:]
X_pro_val = X_product['val'][:]

X_mak_tr = X_maker['train'][:]
X_mak_val = X_maker['val'][:]

X_bra_tr = X_brand['train'][:]
X_bra_val = X_brand['val'][:]

X_mod_tr = X_model['train'][:]
X_mod_val = X_model['val'][:]

X_con_tr = X_con['train'][:]
X_con_val = X_con['val'][:]

In [ ]:
!free

In [9]:
X_pro_d_tr = X_pro_tr[d_train_removed[:] == 0]
X_mak_d_tr = X_mak_tr[d_train_removed[:] == 0]
X_bra_d_tr = X_bra_tr[d_train_removed[:] == 0]
X_mod_d_tr = X_mod_tr[d_train_removed[:] == 0]
X_con_d_tr = X_con_tr[d_train_removed[:] == 0]

print(X_pro_d_tr.shape)
print(X_mak_d_tr.shape)
print(X_bra_d_tr.shape)
print(X_mod_d_tr.shape)
print(X_con_d_tr.shape)

(607560, 32)
(607560, 32)
(607560, 32)
(607560, 32)
(607560, 32)


In [10]:
X_pro_d_val = X_pro_val[d_val_removed[:] == 0]
X_mak_d_val = X_mak_val[d_val_removed[:] == 0]
X_bra_d_val = X_bra_val[d_val_removed[:] == 0]
X_mod_d_val = X_mod_val[d_val_removed[:] == 0]
X_con_d_val = X_con_val[d_val_removed[:] == 0]

print(X_pro_d_val.shape)
print(X_mak_d_val.shape)
print(X_bra_d_val.shape)
print(X_mod_d_val.shape)
print(X_con_d_val.shape)

(150332, 32)
(150332, 32)
(150332, 32)
(150332, 32)
(150332, 32)


In [11]:
with h5py.File('dcate/X_product_rm_d.h5py', 'w') as write:
    write.create_dataset('train', (607560, 32), dtype='i4')
    write.create_dataset('val', (150332, 32), dtype='i4')
    
    write['train'][:] = X_pro_d_tr[:]
    write['val'][:] = X_pro_d_val[:]
    
with h5py.File('dcate/X_maker_rm_d.h5py', 'w') as write:
    write.create_dataset('train', (607560, 32), dtype='i4')
    write.create_dataset('val', (150332, 32), dtype='i4')
    
    write['train'][:] = X_mak_d_tr[:]
    write['val'][:] = X_mak_d_val[:]
    
with h5py.File('dcate/X_brand_rm_d.h5py', 'w') as write:
    write.create_dataset('train', (607560, 32), dtype='i4')
    write.create_dataset('val', (150332, 32), dtype='i4')
    
    write['train'][:] = X_bra_d_tr[:]
    write['val'][:] = X_bra_d_val[:]
    
with h5py.File('dcate/X_model_rm_d.h5py', 'w') as write:
    write.create_dataset('train', (607560, 32), dtype='i4')
    write.create_dataset('val', (150332, 32), dtype='i4')
    
    write['train'][:] = X_mod_d_tr[:]
    write['val'][:] = X_mod_d_val[:]
    
with h5py.File('dcate/X_concate_rm_d.h5py', 'w') as write:
    write.create_dataset('train', (607560, 32), dtype='i4')
    write.create_dataset('val', (150332, 32), dtype='i4')
    
    write['train'][:] = X_con_d_tr[:]
    write['val'][:] = X_con_d_val[:]

In [ ]:
X_pro_s_tr = X_pro_tr[s_train_removed[:] == 0]
X_pro_s_tr.shape

In [ ]:
X_mak_s_tr = X_mak_tr[s_train_removed[:] == 0]
X_mak_s_tr.shape

In [ ]:
X_bra_s_tr = X_bra_tr[s_train_removed[:] == 0]
X_bra_s_tr.shape

In [ ]:
X_mod_s_tr = X_mod_tr[s_train_removed[:] == 0]
X_mod_s_tr.shape

In [ ]:
X_pro_s_val = X_pro_val[s_val_removed[:] == 0]
X_pro_s_val.shape

In [ ]:
X_mak_s_val = X_mak_val[s_val_removed[:] == 0]
X_mak_s_val.shape

In [ ]:
X_bra_s_val = X_bra_val[s_val_removed[:] == 0]
X_bra_s_val.shape

In [ ]:
X_mod_s_val = X_mod_val[s_val_removed[:] == 0]
X_mod_s_val.shape

In [ ]:
with h5py.File('./origin/X_product_for_s.h5py', 'w') as write:
    write.create_dataset('train', (5055885, 32), dtype='i4')
    write.create_dataset('val', (1212813, 32), dtype='i4')
    
    write['train'][:] = X_pro_s_tr[:]
    write['val'][:] = X_pro_s_val[:]

In [ ]:
with h5py.File('./origin/X_maker_for_s.h5py', 'w') as write:
    write.create_dataset('train', (5055885, 32), dtype='i4')
    write.create_dataset('val', (1212813, 32), dtype='i4')
    
    write['train'][:] = X_mak_s_tr[:]
    write['val'][:] = X_mak_s_val[:]

In [ ]:
with h5py.File('./origin/X_brand_for_s.h5py', 'w') as write:
    write.create_dataset('train', (5055885, 32), dtype='i4')
    write.create_dataset('val', (1212813, 32), dtype='i4')
    
    write['train'][:] = X_bra_s_tr[:]
    write['val'][:] = X_bra_s_val[:]

In [ ]:
with h5py.File('./origin/X_model_for_s.h5py', 'w') as write:
    write.create_dataset('train', (5055885, 32), dtype='i4')
    write.create_dataset('val', (1212813, 32), dtype='i4')
    
    write['train'][:] = X_mod_s_tr[:]
    write['val'][:] = X_mod_s_val[:]

In [15]:
X_image_s = h5py.File('./origin/X_image_for_s.h5py', 'r')
X_image_d = h5py.File('./origin/X_image_for_d.h5py', 'r')

print("{} {}".format(X_image_s['train'].shape, X_image_s['val'].shape))
print("{} {}".format(X_image_d['train'].shape, X_image_d['val'].shape))

(5055885, 2048) (1212813, 2048)
(607560, 2048) (150332, 2048)


In [ ]:
cnt = 0
for i in range(ss.shape[0]):
    if ss[i] == 1 :
        cnt += 1
    if (i % 100000 == 0):
        print(i)

print(cnt)

In [ ]:
max_features = 100000
max_len = 32
embed_size = 128
cate_size = 552

In [ ]:
!free

In [ ]:
batch_size = 4096*8

In [ ]:
model = generate_model(max_features, max_len, embed_size, cate_size)

In [ ]:
tf.keras.backend.clear_session()

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

In [ ]:
# tpu_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
tpu_model.compile(loss='categorical_crossentropy', optimizer=tf.train.AdamOptimizer(), metrics=['accuracy'])

In [ ]:
train_generator = data_generator([X_product['train'], X_model['train'], X_brand['train'], X_maker['train'], X_image['train']], y_m_cate['train'], batch_size, len(X_product['train']))
valid_generator = data_generator([X_product['val'], X_model['val'], X_brand['val'], X_maker['val'],   X_image['val']], y_m_cate['val'], batch_size, len(X_product['val']))

In [ ]:
# early_stopping = EarlyStopping(monitor='acc', mode = 'max',patience=5, verbose=1)

model_checkpoint = ModelCheckpoint('b_model_only.h5',monitor='acc', 
                                   mode = 'max', save_best_only=True, verbose=1)
# # reduce_lr = ReduceLROnPlateau(monitor='acc', mode = 'max',factor=0.5, patience=5, min_lr=0.0001, verbose=1)

In [ ]:
tpu_model.fit_generator(generator=train_generator, 
                        steps_per_epoch=int(6507854/batch_size)+1, 
                        validation_data=valid_generator, 
                        validation_steps=int(1626964/batch_size)+1, epochs=10,
                        callbacks=[model_checkpoint])

# M model training!

In [ ]:
for i in load_b.layers:
    print(i.input)

In [ ]:
load_b.layers[-6].output

In [ ]:
load_b = load_model('tpu_b_model.h5')

In [ ]:
load_b.trainable = False

In [ ]:
new_m_input = Input(shape=(1,), name='m_input')
new_m = Dense(256, activation='relu', name='dense_1_m1')(new_m_input)
concate_m = Concatenate(name='concate_m1')([load_b.layers[-6].output, new_m])
# new_m = BatchNormalization()(concate_m)
new_m = Dense(512, activation='relu', name='dense_2_m1')(concate_m)
new_m = Dropout(0.2, name='drop_2')(new_m)
new_m = Dense(256, activation='relu', name='dense_4_m1')(new_m)
new_m = Dropout(0.2, name='drop_3')(new_m)
output = Dense(552, activation='softmax', name='dense_3_m1')(new_m)

model_m = Model([load_b.layers[4].input,load_b.layers[5].input,load_b.layers[6].input,load_b.layers[7].input,load_b.layers[8].input,  new_m_input], output)

In [ ]:
def data_generator_m(list_X,input_cate, y, batch_size, total_size):
    start_idx = 0
    end_idx = 0
    while True:
        if (start_idx + batch_size) >= total_size:
            start_idx = 0
            end_idx = 0
            
        end_idx += batch_size
#         offset = np.random.randint(0, total_size - batch_size)
        batch_X = []
        for X in list_X:
            batch_X.append(np.array(X[start_idx:end_idx]))
            
        cate_input = np.argmax(input_cate[start_idx:end_idx], axis=1)+1
        batch_X.append(cate_input)    
        batch_y = np.array(y[start_idx:end_idx])
        start_idx = end_idx        
        
        yield batch_X, batch_y

In [ ]:
train_generator_m = data_generator_m([X_product['train'], X_model['train'], X_brand['train'], X_maker['train'], X_image['train']],y_b_cate['train'], y_m_cate['train'], batch_size, len(X_product['train']))
valid_generator_m = data_generator_m([X_product['val'], X_model['val'], X_brand['val'], X_maker['val'],   X_image['val']],y_b_cate['val'], y_m_cate['val'], batch_size, len(X_product['val']))

In [ ]:
next(train_generator_m)[0][-1][20]

In [ ]:
model_m.compile(loss='categorical_crossentropy', optimizer=tf.train.AdamOptimizer(), metrics=['acc'])
# early_stopping = EarlyStopping(monitor='acc', mode = 'max',patience=5, verbose=1)

ft_checkpoint = ModelCheckpoint('m_model_transfer.h5',monitor='acc', 
                                   mode = 'max', save_best_only=True, verbose=1)
# # reduce_lr = ReduceLROnPlateau(monitor='acc', mode = 'max',factor=0.5, patience=5, min_lr=0.0001, verbose=1)

In [ ]:
model_m.fit_generator(generator=train_generator_m, 
                        steps_per_epoch=int(6507854/batch_size)+1, 
                        validation_data=valid_generator_m, 
                        validation_steps=int(1626964/batch_size)+1, epochs=10,
                        callbacks=[ft_checkpoint])

In [ ]:
model_m.save('tpu_m_transfer.h5')

# S_transfer model test!

In [ ]:
load_m = load_model('m_model_transfer.h5')
load_m.trainable = False

In [ ]:
print(load_m.layers[0].input)
print(load_m.layers[1].input)
print(load_m.layers[2].input)
print(load_m.layers[3].input)
print(load_m.layers[8].input)
print(load_m.layers[-9].input)
print(load_m.layers[-6].output)

In [ ]:
new_s = Input(shape=(1,), name='input_s')
new1 = Dense(256, activation='relu')(new_s)
concate_m = Concatenate()([load_m.layers[-6].output, new1])
# new1 = BatchNormalization()(concate_m)
new1 = Dense(256, activation='relu')(concate_m)
new1 = Dropout(0.2)(new1)
output = Dense(3191, activation='softmax')(new1)

model_s = Model([load_m.layers[0].input,load_m.layers[1].input,load_m.layers[2].input,load_m.layers[3].input,load_m.layers[8].input, load_m.layers[-9].input, new_s], output)

In [ ]:
max_features = 100000
max_len = 32
embed_size = 128
cate_size = 3191

In [ ]:
def data_generator_s(list_X,input_cate_b, input_cate_m, y, batch_size, total_size):
#     start_idx = 0
#     end_idx = 0
    while True:
#         end_idx += batch_size
        offset = np.random.randint(0, total_size - batch_size)
        batch_X = []
        for X in list_X:
            batch_X.append(np.array(X[offset:offset+batch_size]))
            
        cate_input_b = np.argmax(input_cate_b[offset:offset+batch_size], axis=1)+1
        batch_X.append(cate_input_b)
        cate_input_m = np.argmax(input_cate_m[offset:offset+batch_size], axis=1)+1
        batch_X.append(cate_input_m)
        batch_y = np.array(y[offset:offset+batch_size])
#         start_idx = end_idx        
        
        yield batch_X, batch_y

In [ ]:
train_generator_s = data_generator_s([X_product['train'], X_model['train'], X_brand['train'], X_maker['train'], X_image['train']],y_b_cate['train'], y_m_cate['train'],y_s_cate['train'], batch_size, len(X_product['train']))
valid_generator_s = data_generator_s([X_product['val'], X_model['val'], X_brand['val'], X_maker['val'],   X_image['val']],y_b_cate['val'], y_m_cate['val'], y_s_cate['val'], batch_size, len(X_product['val']))

In [ ]:
model_s.compile(loss='categorical_crossentropy', optimizer=tf.train.AdamOptimizer(), metrics=['acc'])
# early_stopping = EarlyStopping(monitor='acc', mode = 'max',patience=5, verbose=1)

s_checkpoint = ModelCheckpoint('s_model_transfer.h5',monitor='acc', 
                                   mode = 'max', save_best_only=True, verbose=1)
# # reduce_lr = ReduceLROnPlateau(monitor='acc', mode = 'max',factor=0.5, patience=5, min_lr=0.0001, verbose=1)

In [ ]:
model_s.fit_generator(generator=train_generator_s, 
                        steps_per_epoch=int(6507854/batch_size)+1, 
                        validation_data=valid_generator_s, 
                        validation_steps=int(1626964/batch_size)+1, epochs=10,
                        callbacks=[s_checkpoint])

# D model test!

In [ ]:
load_d = load_model('s_model_transfer.h5')
load_d.trainable = False

In [ ]:
for i in load_d.layers:
    print(i.input)

In [ ]:
print(load_d.layers[4].input)
print(load_d.layers[5].input)
print(load_d.layers[6].input)
print(load_d.layers[7].input)
print(load_d.layers[8].input)
print(load_d.layers[-10].input)
print(load_d.layers[-7].input)
print(load_d.layers[-4].output)

In [ ]:
new_d = Input(shape=(1,), name='input_d')
new1 = Dense(256, activation='relu')(new_d)
concate_d = Concatenate()([load_d.layers[-4].output, new1])
# new1 = BatchNormalization()(concate_m)
new1 = Dense(256, activation='relu')(concate_d)
new1 = Dropout(0.2)(new1)
output = Dense(405, activation='softmax')(new1)

model_d = Model([load_d.layers[4].input,load_d.layers[5].input,load_d.layers[6].input,load_d.layers[7].input,load_d.layers[8].input, load_d.layers[-10].input, load_d.layers[-7].input, new_d], output)

In [ ]:
def data_generator_d(list_X,input_cate_b, input_cate_m, input_cate_s, y, batch_size, total_size):
    start_idx = 0
    end_idx = 0
    while True:
        if (start_idx + batch_size) >= total_size:
            start_idx = 0
            end_idx = 0
        end_idx += batch_size
#         offset = np.random.randint(0, total_size - batch_size)
        batch_X = []
        for X in list_X:
            batch_X.append(np.array(X[start_idx:end_idx]))
            
        cate_input_b = np.argmax(input_cate_b[start_idx:end_idx], axis=1)+1
        batch_X.append(cate_input_b)
        cate_input_m = np.argmax(input_cate_m[start_idx:end_idx], axis=1)+1
        batch_X.append(cate_input_m)
        cate_input_s = np.argmax(input_cate_s[start_idx:end_idx], axis=1)
        batch_X.append(cate_input_s)
        batch_y = np.array(y[start_idx:end_idx])
        start_idx = end_idx        
        
        yield batch_X, batch_y

In [ ]:
batch_size=4096*8

In [ ]:
train_generator_d = data_generator_d([X_product['train'], X_model['train'], X_brand['train'], X_maker['train'], X_image['train']],y_b_cate['train'], y_m_cate['train'],y_s_cate['train'], y_d_cate['train'], batch_size, len(X_product['train']))
valid_generator_d = data_generator_d([X_product['val'], X_model['val'], X_brand['val'], X_maker['val'], X_image['val']],y_b_cate['val'], y_m_cate['val'], y_s_cate['val'], y_d_cate['val'], batch_size, len(X_product['val']))

In [ ]:
model_d.compile(loss='categorical_crossentropy', optimizer=tf.train.AdamOptimizer(), metrics=['acc'])
# early_stopping = EarlyStopping(monitor='acc', mode = 'max',patience=5, verbose=1)

d_checkpoint = ModelCheckpoint('d_model_transfer.h5',monitor='acc', 
                                   mode = 'max', save_best_only=True, verbose=1)
# # reduce_lr = ReduceLROnPlateau(monitor='acc', mode = 'max',factor=0.5, patience=5, min_lr=0.0001, verbose=1)

In [ ]:
model_d.fit_generator(generator=train_generator_d, 
                        steps_per_epoch=int(6507854/batch_size)+1, 
                        validation_data=valid_generator_d, 
                        validation_steps=int(1626964/batch_size)+1, epochs=10,
                        callbacks=[d_checkpoint])